In [1]:
import sqlite3
import pandas as pd

from IPython.display import display

In [2]:
def create_placeholder(n):
    p = ",".join(["?"] * n)
    return "(" + p + ")"

In [3]:
%%time

conn = sqlite3.connect("../aminer.db")
df1 = pd.read_sql_query(
    """
    SELECT ACM.paper_index, ACM.paper_title, ACM.year, ACM.abstract, count(AA.author_name) as NUM_AUTHORS 
    FROM ACM
    LEFT JOIN ACM_AUTHOR AS AA
    ON ACM.paper_index = AA.paper_index
    GROUP BY ACM.paper_index
    HAVING NUM_AUTHORS > 10
    ORDER BY count(AA.author_name) DESC
    """, 
    conn
)

CPU times: user 10.8 s, sys: 675 ms, total: 11.5 s
Wall time: 11.5 s


In [4]:
display(df1.head())

,paper_index,paper_title,year,abstract,NUM_AUTHORS
0,569904,An overview of the BlueGene/L Supercomputer,2002,This paper gives an overview of the BlueGene/L...,115
1,1479158,Deployment of the ATLAS high level trigger,2005,The ATLAS combined test beam in the second hal...,110
2,785222,The Grid2003 Production Grid: Principles and P...,2004,The Grid2003 Project has deployed a multi-virt...,102
3,1202293,Interoperation of world-wide production e-Scie...,2009,Many production Grid and e-Science infrastruct...,88
4,1479229,Implementation and performance of the seeded r...,2005,ATLAS is one of the four LHC experiments that ...,83


In [5]:
%%time

conn = sqlite3.connect("../aminer.db")
df2 = pd.read_sql_query(
    """
    SELECT ACM.paper_index, ACM.paper_title, ACM.year, COUNT(AC.paper_index) as CITATIONS 
    FROM ACM_CITATION AS AC
    LEFT JOIN ACM
    ON AC.reference_id = ACM.paper_index
    GROUP BY AC.reference_id
    ORDER BY CITATIONS DESC
    """, 
    conn
)


CPU times: user 25 s, sys: 10.4 s, total: 35.5 s
Wall time: 35.5 s


In [6]:
display(df2.head())

,paper_index,paper_title,year,CITATIONS
0,2135000,INFORMS Journal on Computing,2014,938039
1,2381731,Effects of Terms Recognition Mistakes on Reque...,2012,82080
2,408395,Computers and Intractability: A Guide to the T...,1979,8620
3,369235,"Genetic Algorithms in Search, Optimization and...",1989,6526
4,190580,The nature of statistical learning theory,1995,5297


In [27]:
paper_titles = [
    "Header and Unit Inference for Spreadsheets Through Spatial Analyses", 
    "Schema extraction for tabular data on the web"
]

In [34]:
indices = pd.read_sql_query(
    """
    SELECT paper_index FROM ACM WHERE paper_title in {}
    """.format(create_placeholder(len(papers))), 
    params = paper_titles, 
    con = conn
)["paper_index"].values.tolist()

print(indices)

[786387, 2030591]


In [35]:
links = []

In [38]:
%%time

for idx in indices:
    cites = pd.read_sql_query(
        """
        SELECT ACM.paper_index FROM ACM 
        LEFT JOIN ACM_CITATION AS AC
        ON ACM.paper_index = AC.paper_index
        WHERE AC.reference_id = (?)
        """, 
        params = (idx, ), 
        con = conn
    )["paper_index"].values.tolist()

    for cite in cites:
        links.append([index, cite])
        
indices = cites
print(len(links))

53
CPU times: user 36.7 s, sys: 2.26 s, total: 38.9 s
Wall time: 39.2 s
